<a href="https://colab.research.google.com/github/Dkathayat/mini_chatgpt_01/blob/main/tiny_chat_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-05-18 04:54:09--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2024-05-18 04:54:09 (26.7 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [ ]:
# reading the data
with open('input.txt','r',encoding='utf-8') as f:
  text = f.read()

In [ ]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [ ]:
# printing unique char in the data set
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [ ]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hello World!"))
print(decode(encode("hello world!")))

[46, 43, 50, 50, 53, 1, 35, 53, 56, 50, 42, 2]
hello world!


In [ ]:
#importing torch and encoding all the char in the text
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape,data.dtype)

torch.Size([1115394]) torch.int64


In [ ]:
#splitting the data for training and testing
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  # print(target)
  # print(target.dtype)

In [ ]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size] for i in ix])
  return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('tragents:')
print(yb.shape)
print(yb)

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    # target = yb[b,t]
    # print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
tragents:
torch.Size([4, 7])
tensor([[43, 58,  5, 57,  1, 46, 43],
        [53, 56,  1, 58, 46, 39, 58],
        [58,  1, 58, 46, 39, 58,  1],
        [17, 27, 10,  0, 21,  1, 54]])


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table =  nn.Embedding(vocab_size, vocab_size)

  def forward(self,idx,targets):
    logits = self.token_embedding_table(idx)

    return logits

m = BigramLanguageModel(vocab_size)
out = m(xb, yb)
print(out.shape)


torch.Size([4, 8, 65])
